<a href="https://colab.research.google.com/github/ThomasZ89/Rent_Pred2/blob/master/Notebooks/Folium_FFM_Rent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geopandas
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import folium

In [ ]:
plz_map = pd.read_excel (r"/content/plz_mapping.xlsx")
plz_map["plz"] = plz_map["plz"].astype(str)

In [ ]:
#geojson  = gpd.read_file("plz-gebiete.json")
#gdf.to_file("output.json", driver="GeoJSON")
#with open('/content/output.json') as json_file:
#    data = json.load(json_file)
#data = '/content/output.json'

In [ ]:
PLZ_coeff = pd.read_csv("/content/PLZ_param.csv")
PLZ_coeff.columns = ["Name","coeff","plz"]
PLZ_coeff["plz"] = PLZ_coeff["plz"].astype(str)
PLZ_coeff["coeff"] = PLZ_coeff["coeff"].astype(int)

In [ ]:
plz_shape_df = gpd.read_file('/content/plz-gebiete.shp')

In [ ]:
gdf = pd.merge(
    left=plz_shape_df, 
    right=PLZ_coeff, 
    on='plz',
    how='inner'
)

In [ ]:
gdf = pd.merge(
    left=gdf, 
    right=plz_map, 
    on='plz',
    how='left'
)

In [ ]:
gdf['centroid_lon'] = gdf['geometry'].centroid.x
gdf['centroid_lat'] = gdf['geometry'].centroid.y

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [ ]:
m = folium.Map(location=[np.median(gdf['centroid_lat'].tolist()),
                         np.median(gdf['centroid_lon'].tolist())],
               tiles='Stamen Toner', zoom_start=12)

## add chloropleth layer
m.choropleth(
    geo_data=gdf,
    name='Rent coefficient',
    data=gdf,
    columns=['plz', 'coeff',"name"],
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.5,
    key_on='feature.properties.plz',
    legend_name='Rent Price coefficient'
)  

# add markers with basic information
fg = folium.FeatureGroup(name='District Info')
for lat, lon, val, name in zip(gdf['centroid_lat'].tolist(),
                               gdf['centroid_lon'].tolist(),
                               gdf['coeff'].tolist(),
                               gdf['Stadtteil'].tolist()):
    html = f"""
    <h2>{name}<\h2><br>
    <h4>Coefficient is {int(round(val,0))} <\h4>
    """
    fg.add_child(folium.Marker(location=[lat, lon], popup=html))

m.add_child(fg)

# enable layers to be turned in or out
folium.LayerControl().add_to(m)

/usr/local/lib/python3.6/dist-packages/folium/folium.py:426: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [ ]:
m

In [ ]:
m.save('map.html')

In [ ]:
type(m)

folium.folium.Map